In [1]:
import pandas as pd
from pathlib import Path
import tensorflow as tf

# Step 1: Preprocess the Data

In [2]:
# Load data into DataFrame
file_path = Path('Resources/charity_data.csv')
df = pd.read_csv(file_path)
df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


Q: What variable(s) are the target(s) for your model? A: TThe target is the IS_SUCCESSFUL column to determine if a venture is succesful after recieving funding.

Q: What variable(s) are the feature(s) for your model? A: APPLICATION_TYPE, AFFILIATION, CLASSIFICATION, USE_CASE, ORGANIZATION, STATUS, INCOME_AMT, SPECIAL_CONSIDERATIONS, ASK_AMT

In [3]:
# Drop the EIN and NAME columns.
df.drop(['EIN'], axis = 1,inplace = True)

df.head()

,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [4]:
df.drop(['NAME'], axis = 1,inplace = True)

df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# Determine the number of unique values for each column
for column in df:
    print(f'{column}: {df[column].nunique()}')

APPLICATION_TYPE: 17
AFFILIATION: 6
CLASSIFICATION: 71
USE_CASE: 5
ORGANIZATION: 4
STATUS: 2
INCOME_AMT: 9
SPECIAL_CONSIDERATIONS: 2
ASK_AMT: 8747
IS_SUCCESSFUL: 2


In [6]:
#Binning rare categorical variables for columns that have more than 10 unique values
df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [7]:
def application_bin(type):
    if type in ['T13','T12','T2','T25','T14','T29','T15','T17']:
        return 'Other'
    else:
        return type

In [8]:
df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].apply(application_bin)

In [9]:
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
T9         156
Other      120
Name: APPLICATION_TYPE, dtype: int64

In [10]:
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [11]:
def classification_bin(classification):
    if classification in ['C1000', 'C2000', 'C1200', 'C3000', 'C2100', 'C7000', 'C1700', 'C4000','C5000']:
        return classification
    else:
        return 'Other'

In [12]:
df['CLASSIFICATION'] = df['CLASSIFICATION'].apply(classification_bin)

In [13]:
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other      887
C7000      777
C1700      287
C4000      194
C5000      116
Name: CLASSIFICATION, dtype: int64

In [14]:
# Use get_dummies to encode categorical variables
df = pd.get_dummies(df)
df

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [15]:
X = df.drop(['IS_SUCCESSFUL'], axis = 1)
y = df['IS_SUCCESSFUL']
X

,STATUS,ASK_AMT,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,0,0,0,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,0,0,0,1,0,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,0,0,0,1,0,0,0,0,...,0,0,1,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34295,1,5000,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34296,1,5000,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
34297,1,5000,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0


In [16]:
# Scaling X variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

X_scaler = StandardScaler().fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Step 2: Compile, Train, and Evaluate the Model

In [18]:
# Create a neural network model by assigning the number of input features and nodes for each layer using TensorFlow and Keras
number_input_features = len(X_train_scaled[0])

nn_model = tf.keras.models.Sequential()

# Create the first hidden layer and choose an appropriate activation function
nn_model.add(tf.keras.layers.Dense(units = 1, activation = "relu", input_dim = number_input_features))

# Create an output layer with an appropriate activation function
nn_model.add(tf.keras.layers.Dense(units = 1, activation = "sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 1)                 49        
                                                                 
 dense_1 (Dense)             (None, 1)                 2         
                                                                 
Total params: 51
Trainable params: 51
Non-trainable params: 0
_________________________________________________________________


In [19]:
# Compile and train the model
nn_model.compile(loss="binary_crossentropy", optimizer = "adam", metrics = ["accuracy"])

fit_model = nn_model.fit(X_train_scaled, y_train, epochs = 100)

Epoch 1/100
804/804 [==============================] - 2s 1ms/step - loss: 0.6836 - accuracy: 0.5714
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.6190 - accuracy: 0.6990
Epoch 3/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5968 - accuracy: 0.7050
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5857 - accuracy: 0.7139
Epoch 5/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5743 - accuracy: 0.7238
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5731 - accuracy: 0.7227
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5727 - accuracy: 0.7231
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5725 - accuracy: 0.7221
Epoch 9/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5722 - accuracy: 0.7230
Epoch 10/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5721 - accura

804/804 [==============================] - 1s 1ms/step - loss: 0.5709 - accuracy: 0.7236
Epoch 82/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5710 - accuracy: 0.7244
Epoch 83/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5710 - accuracy: 0.7236
Epoch 84/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5709 - accuracy: 0.7235
Epoch 85/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5709 - accuracy: 0.7231
Epoch 86/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5709 - accuracy: 0.7234
Epoch 87/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5710 - accuracy: 0.7236
Epoch 88/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5709 - accuracy: 0.7239
Epoch 89/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5710 - accuracy: 0.7237
Epoch 90/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5710 - accuracy: 

In [24]:
# Evaluate the model using the test data to determine the loss and accuracy
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test)
print(f'Model Loss: {model_loss}, Model Accuracy: {model_accuracy}')

268/268 [==============================] - 0s 1ms/step - loss: 0.5738 - accuracy: 0.7186
Model Loss: 0.5737531185150146, Model Accuracy: 0.7186005711555481
